In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np

import pickle

labels = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
img_size = 256
def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        #class_num = label # edit1
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num]) # edit1
            except Exception as e:
                print(e)
    return np.array(data)

train = get_data('spectrogram/train')
val = get_data('spectrogram/test')

x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in train:
  x_train.append(feature)
  y_train.append(label)

for feature, label in val:
  x_val.append(feature)
  y_val.append(label)

# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        #rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        #horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(256,256,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(10, activation="softmax"))

print(model.summary())

opt = Adam(lr=0.0001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])


<ipython-input-2-1b9c9e9baad0>:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(data)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0

C:\Users\DELL\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [13]:
# The EXTREMELY time consuming step
history = model.fit(x_train,y_train,epochs = 500, validation_data = (x_val, y_val))

Epoch 1/500


C:\Users\DELL\anaconda3\lib\site-packages\keras\backend.py:4906: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


16/16 [==============================] - 57s 3s/step - loss: 2.3002 - accuracy: 0.1126 - val_loss: 2.2260 - val_accuracy: 0.1275
Epoch 2/500
16/16 [==============================] - 46s 3s/step - loss: 2.1327 - accuracy: 0.1897 - val_loss: 2.1040 - val_accuracy: 0.2250
Epoch 3/500
16/16 [==============================] - 44s 3s/step - loss: 1.9476 - accuracy: 0.2945 - val_loss: 1.9983 - val_accuracy: 0.2725
Epoch 4/500
16/16 [==============================] - 42s 3s/step - loss: 1.8193 - accuracy: 0.3360 - val_loss: 1.9701 - val_accuracy: 0.3000
Epoch 5/500
16/16 [==============================] - 43s 3s/step - loss: 1.7324 - accuracy: 0.3557 - val_loss: 2.0069 - val_accuracy: 0.3075
Epoch 6/500
16/16 [==============================] - 44s 3s/step - loss: 1.7051 - accuracy: 0.3992 - val_loss: 1.9791 - val_accuracy: 0.3075
Epoch 7/500
16/16 [==============================] - 42s 3s/step - loss: 1.6389 - accuracy: 0.4209 - val_loss: 2.0876 - val_accuracy: 0.2550
Epoch 8/500
16/16 [======

Epoch 59/500
16/16 [==============================] - 43s 3s/step - loss: 0.3436 - accuracy: 0.9032 - val_loss: 1.9433 - val_accuracy: 0.4375
Epoch 60/500
16/16 [==============================] - 42s 3s/step - loss: 0.2994 - accuracy: 0.9368 - val_loss: 1.9101 - val_accuracy: 0.4400
Epoch 61/500
16/16 [==============================] - 43s 3s/step - loss: 0.3299 - accuracy: 0.8913 - val_loss: 1.9903 - val_accuracy: 0.4225
Epoch 62/500
16/16 [==============================] - 43s 3s/step - loss: 0.2998 - accuracy: 0.9209 - val_loss: 1.9600 - val_accuracy: 0.4575
Epoch 63/500
16/16 [==============================] - 42s 3s/step - loss: 0.2890 - accuracy: 0.9348 - val_loss: 1.9910 - val_accuracy: 0.4425
Epoch 64/500
16/16 [==============================] - 43s 3s/step - loss: 0.2926 - accuracy: 0.9209 - val_loss: 2.1334 - val_accuracy: 0.4225
Epoch 65/500
16/16 [==============================] - 43s 3s/step - loss: 0.2957 - accuracy: 0.9170 - val_loss: 2.1573 - val_accuracy: 0.4425
Epoch 

16/16 [==============================] - 74s 5s/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 3.0273 - val_accuracy: 0.4725
Epoch 174/500
16/16 [==============================] - 73s 5s/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 2.9785 - val_accuracy: 0.4825
Epoch 175/500
16/16 [==============================] - 71s 5s/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 2.9524 - val_accuracy: 0.4900
Epoch 176/500
16/16 [==============================] - 74s 5s/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 3.0636 - val_accuracy: 0.4900
Epoch 177/500
16/16 [==============================] - 74s 5s/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 3.0538 - val_accuracy: 0.4725
Epoch 178/500
16/16 [==============================] - 73s 5s/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 3.0287 - val_accuracy: 0.4875
Epoch 179/500
16/16 [==============================] - 73s 5s/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 3.0391 - val_accuracy: 0.4750
Epoch 180/500

16/16 [==============================] - 47s 3s/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 3.5665 - val_accuracy: 0.4850
Epoch 289/500
16/16 [==============================] - 49s 3s/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 3.5219 - val_accuracy: 0.4825
Epoch 290/500
16/16 [==============================] - 47s 3s/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 3.5576 - val_accuracy: 0.4850
Epoch 291/500
16/16 [==============================] - 47s 3s/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 3.6081 - val_accuracy: 0.4800
Epoch 292/500
16/16 [==============================] - 47s 3s/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 3.5366 - val_accuracy: 0.4850
Epoch 293/500
16/16 [==============================] - 46s 3s/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 3.5801 - val_accuracy: 0.4825
Epoch 294/500
16/16 [==============================] - 47s 3s/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 3.5941 - val_accuracy: 0.4825
Epoch 295/500

Epoch 402/500
16/16 [==============================] - 42s 3s/step - loss: 8.2121e-04 - accuracy: 1.0000 - val_loss: 3.3835 - val_accuracy: 0.4825
Epoch 403/500
16/16 [==============================] - 42s 3s/step - loss: 6.7050e-04 - accuracy: 1.0000 - val_loss: 3.4046 - val_accuracy: 0.4850
Epoch 404/500
16/16 [==============================] - 43s 3s/step - loss: 7.2066e-04 - accuracy: 1.0000 - val_loss: 3.4089 - val_accuracy: 0.4875
Epoch 405/500
16/16 [==============================] - 43s 3s/step - loss: 7.5359e-04 - accuracy: 1.0000 - val_loss: 3.3933 - val_accuracy: 0.4925
Epoch 406/500
16/16 [==============================] - 49s 3s/step - loss: 6.9450e-04 - accuracy: 1.0000 - val_loss: 3.4278 - val_accuracy: 0.4950
Epoch 407/500
16/16 [==============================] - 39s 2s/step - loss: 7.0943e-04 - accuracy: 1.0000 - val_loss: 3.4431 - val_accuracy: 0.4850
Epoch 408/500
16/16 [==============================] - 51s 3s/step - loss: 7.5931e-04 - accuracy: 1.0000 - val_loss: 3

Epoch 458/500
16/16 [==============================] - 52s 3s/step - loss: 3.7265e-04 - accuracy: 1.0000 - val_loss: 3.6262 - val_accuracy: 0.4925
Epoch 459/500
16/16 [==============================] - 52s 3s/step - loss: 4.5581e-04 - accuracy: 1.0000 - val_loss: 3.6760 - val_accuracy: 0.4825
Epoch 460/500
16/16 [==============================] - 55s 4s/step - loss: 3.9971e-04 - accuracy: 1.0000 - val_loss: 3.6910 - val_accuracy: 0.4850
Epoch 461/500
16/16 [==============================] - 59s 4s/step - loss: 4.0304e-04 - accuracy: 1.0000 - val_loss: 3.6212 - val_accuracy: 0.4950
Epoch 462/500
16/16 [==============================] - 63s 4s/step - loss: 3.3453e-04 - accuracy: 1.0000 - val_loss: 3.6050 - val_accuracy: 0.4975
Epoch 463/500
16/16 [==============================] - 60s 4s/step - loss: 3.9198e-04 - accuracy: 1.0000 - val_loss: 3.6339 - val_accuracy: 0.4950
Epoch 464/500
16/16 [==============================] - 48s 3s/step - loss: 3.4392e-04 - accuracy: 1.0000 - val_loss: 3

In [14]:
model.save("fullModelSave")

INFO:tensorflow:Assets written to: fullModelSave\assets


In [3]:
# definition of wavelet & spectrogram generating functions respectively (for one audio tape at a time)

def waveletGen(cls): # all testing in deployTestDirectory, test audio wavelet saved as 1.png, cls is the filename with .mp3
    
    x, sr = librosa.load('deployTest/testAudioClip' + '/' + cls)
    # plt.figure(figsize=(14, 5))
    librosa.display.waveplot(x)
    plt.savefig('deployTest/testWavelet/' + '1' + '.png')
    plt.close()
    
    '''
    img_names = os.listdir('genres/' + cls)
    os.makedirs('wavelets/train/' + cls)
    os.makedirs('wavelets/test/' + cls)
    print(cls)
    train_names = img_names[:60]
    test_names = img_names[60:]
    cnt = 0
    for nm in train_names:
        cnt += 1
        x, sr = librosa.load('genres/' + cls + '/' + nm)
        # plt.figure(figsize=(14, 5))
        librosa.display.waveplot(x)
        plt.savefig('wavelets/train/' + cls + '/' + str(cnt) + '.png')
        plt.close()

    cnt = 0
    for nm in test_names:
        cnt += 1
        x, sr = librosa.load('genres/' + cls + '/' + nm)
        # plt.figure(figsize=(14, 5))
        librosa.display.waveplot(x)
        plt.savefig('wavelets/test/' + cls + '/' + str(cnt) + '.png')
        plt.close()
        
    '''


def spectrogramGen(cls): # all testing in deployTestDirectory, test audio spectrogram saved as 1.png, cls is the filename with .mp3
    
    x, sr = librosa.load('deployTest/testAudioClip' + '/' + cls)
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))
    librosa.display.specshow(Xdb)
    plt.savefig('deployTest/testSpectrogram/' + '1' + '.png')
    plt.close()
    
    '''
    img_names = os.listdir('genres/' + cls)
    os.makedirs('spectrogram/train/' + cls)
    os.makedirs('spectrogram/test/' + cls)
    print(cls)
    train_names = img_names[:60]
    test_names = img_names[60:]
    cnt = 0
    for nm in train_names:
        cnt += 1
        x, sr = librosa.load('genres/' + cls + '/' + nm)
        X = librosa.stft(x)
        Xdb = librosa.amplitude_to_db(abs(X))
        librosa.display.specshow(Xdb)
        plt.savefig('spectrogram/train/' + cls + '/' + str(cnt) + '.png')
        plt.close()

    cnt = 0
    for nm in test_names:
        cnt += 1
        x, sr = librosa.load('genres/' + cls + '/' + nm)
        X = librosa.stft(x)
        Xdb = librosa.amplitude_to_db(abs(X))
        librosa.display.specshow(Xdb)
        plt.savefig('spectrogram/test/' + cls + '/' + str(cnt) + '.png')
        plt.close()
    
    '''
        

In [ ]:
# spectrogram & wavelet dataset creation [left off here]

data = []
path = 'deployTest/testSpectrogram'
class_num = labels.index(label) # replace labels with genre name
#class_num = label # edit1
for img in os.listdir(path):
    try:
        img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
        resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
        data.append([resized_arr, class_num]) # edit1
    except Exception as e:
        print(e)
        
valSpectrogram = np.array(data)

data = []
path = 'deployTest/testWavelet'
class_num = labels.index(label) # replace labels with genre name
#class_num = label # edit1
for img in os.listdir(path):
    try:
        img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
        resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
        data.append([resized_arr, class_num]) # edit1
    except Exception as e:
        print(e)
        
valWavelet = np.array(data)

In [4]:
reconstructed_model = load_model("fullModelSave")
predictions = reconstructed_model.predict(np.array(None)) # replace None with value array
print(predictions)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).